<a href="https://colab.research.google.com/github/christianbarreto07/Predict_Spam_or_Ham_Email/blob/main/Email_SPAM_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Carregamos os dados.

In [43]:
import pandas as pd
import numpy as np

In [25]:
data_spam_ham = pd.read_csv('/content/spam_ham_dataset.csv') #Coloco os dados em uma variável e controlo o formato do arquivo com o index_col=[0]

Removendo coluna Unnamed: 0 no próprio data_spam_ham

In [29]:
data_spam_ham.drop(['Unnamed: 0'], axis=1, inplace=True)

Impressão dos dados

In [38]:
data_spam_ham

,label,text,label_num
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...
5166,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,ham,Subject: industrial worksheets for august 2000...,0


Analisando o dataset.

In [44]:
np.unique(data_spam_ham['label_num'], return_counts = True) # 3672 emails legítimos e 1499 spams.

(array([0, 1]), array([3672, 1499]))

In [39]:
data_spam_ham[data_spam_ham['label_num'] == 0] #Emails Legítimos.

,label,text,label_num
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
4,ham,Subject: re : indian springs\r\nthis deal is t...,0
5,ham,Subject: ehronline web address change\r\nthis ...,0
...,...,...,...
5165,ham,"Subject: fw : crosstex energy , driscoll ranch...",0
5166,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0


In [47]:
data_spam_ham[data_spam_ham['label_num'] == 1] #Emails spam.

,label,text,label_num,length
3,spam,"Subject: photoshop , windows , office . cheap ...",1,414
7,spam,Subject: looking for medication ? we ` re the ...,1,784
10,spam,Subject: vocable % rnd - word asceticism\r\nvc...,1,6922
11,spam,Subject: report 01405 !\r\nwffur attion brom e...,1,337
13,spam,Subject: vic . odin n ^ ow\r\nberne hotbox car...,1,395
...,...,...,...,...
5159,spam,Subject: pictures\r\nstreamlined denizen ajar ...,1,452
5161,spam,Subject: penny stocks are about timing\r\nnoma...,1,4158
5162,spam,Subject: anomaly boys from 3881\r\nuosda apapr...,1,482
5164,spam,Subject: slutty milf wants to meet you\r\ntake...,1,69


Importando biblioteca para trabalhar com linguagem natural(Humana) e para extração de informações de texto.

In [12]:
import nltk

Stopwords é um conjunto de palavras que não agregam conhecimento, ou seja, não farão sentidos na analise e deixará o processamento mais custoso. Ex: preposições.

In [56]:
from nltk.corpus import stopwords

Strings para trabalhar com funções e cadeia de palavras.

In [52]:
import string

Importando Sklearn para trabalharmos com o aprendizado de máquina.

In [11]:
from sklearn.model_selection import train_test_split

In [18]:
from sklearn.pipeline import Pipeline

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer

In [20]:
from sklearn.naive_bayes import MultinomialNB

In [73]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
Fazendo download dos pacotes do nltk. (Uma unica vez é necessária chamar essa função)

In [58]:
#nltk.download())
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Criando nova coluna chamada length (Comprimento do subject)

In [46]:
data_spam_ham['length'] = data_spam_ham['text'].apply(len)
data_spam_ham['length']

0        327
1         97
2       2524
3        414
4        336
        ... 
5166     534
5167    1114
5168     402
5169     573
5170    1114
Name: length, Length: 5171, dtype: int64

Analisando o tamanho das mensagens pelo tipo (spam ou ham).

In [ ]:
data_spam_ham.hist(column='length', by='label_num', bins=70,figsize=(15,6)) #Bins para o fator de agrupamento das informações. Figsize para o tamanho

Criando uma função para processar o texto

In [54]:
def processaTexto(texto):
  nopunc = [char for char in texto if char not in string.punctuation] #Remoção de pontuações
  nopunc = ''.join(nopunc) #agrupando o que foi desagrupado com o nopunc.
  CleanWords = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')] #Removendo palavras desnecessárias, separando as strings em sub-strings e transformando todos em letras minúsculas.
  return CleanWords

Aplicando a função processaTexto nos textos do dataset e processando todos os dados.

In [59]:
data_spam_ham['text'].apply(processaTexto)

0       [Subject, enron, methanol, meter, 988291, foll...
1       [Subject, hpl, nom, january, 9, 2001, see, att...
2       [Subject, neon, retreat, ho, ho, ho, around, w...
3       [Subject, photoshop, windows, office, cheap, m...
4       [Subject, indian, springs, deal, book, teco, p...
                              ...                        
5166    [Subject, put, 10, ft, transport, volumes, dec...
5167    [Subject, 3, 4, 2000, following, noms, hpl, ta...
5168    [Subject, calpine, daily, gas, nomination, jul...
5169    [Subject, industrial, worksheets, august, 2000...
5170    [Subject, important, online, banking, alert, d...
Name: text, Length: 5171, dtype: object

Separando os previsores e classe em variáveis.

In [ ]:
X_spam_ham = data_spam_ham.iloc[:, 1].values #Previsores
X_spam_ham

In [67]:
Y_spam_ham = data_spam_ham.iloc[:,2].values #Classe
Y_spam_ham #0 = Ham(Legitimo), 1 = Spam

array([0, 0, 0, ..., 0, 0, 1])

Criando as variáveis de treino e teste para previsores e classe.

In [68]:
X_data_spam_ham_treino, X_data_spam_ham_teste, Y_data_spam_ham_treino, Y_data_spam_ham_teste = train_test_split(X_spam_ham, Y_spam_ham, test_size=0.15, random_state=0)
#Criei as variáveis, chamei a função de train_test_split, passei os paramentros de previsores(x) e classe(y) referente ao dataset, coloquei 15% do dataset para teste e o restante para treino.

Transferindo esses novos conjuntos para o naive bayes usando o conceito de pipeline

In [69]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=processaTexto)), #Contando o Bag Of Words(Saco de palavras) para definir as palavras que são tipicas de spam ou as típicas de ham.
    ('tfidf', TfidfTransformer()), #Faz com que pegue as palavras que aparecem nos dois conjuntos(ham,spam) não seja tão levada em consideração para a analise final e balanceando essas palavras.
    ('classifier', MultinomialNB()) #Aplicando o treinamento de NaiveBayes
])

Começando o treinamento

In [70]:
pipeline.fit(X_data_spam_ham_treino, Y_data_spam_ham_treino) #chamando a função para o treinamento e passando como parametro as variáveis X(previsores) e Y(classe) de treino.

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function processaTexto at 0x7cb42c965c60>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [71]:
predict = pipeline.predict(X_data_spam_ham_teste) #Fazendo a predição

Informações detalhadas sobre os resultados.

In [74]:
accuracy_score(Y_data_spam_ham_teste, predict) #Checando a porcentagem de acerto

0.9162371134020618

In [75]:
classification_report(Y_data_spam_ham_teste, predict) #Mostrando os valores do precision, recall, f1-score, support e accuracy.

'              precision    recall  f1-score   support\n\n           0       0.89      1.00      0.94       550\n           1       1.00      0.71      0.83       226\n\n    accuracy                           0.92       776\n   macro avg       0.95      0.86      0.89       776\nweighted avg       0.93      0.92      0.91       776\n'

In [76]:
confusion_matrix(Y_data_spam_ham_teste, predict)
#Matriz de confusão na qual mostra os verdadeiros posivos e negativos, juntamente com os falsos positivos e negativos

array([[550,   0],
       [ 65, 161]])